## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,33.08,60,100,3.13,overcast clouds
1,1,Kodiak,US,57.7900,-152.4072,29.79,67,100,7.44,overcast clouds
2,2,Ranfurly,NZ,-45.1333,170.1000,69.06,56,43,0.91,scattered clouds
3,3,Mataura,NZ,-46.1927,168.8643,68.49,51,68,0.65,broken clouds
4,4,Barrow,US,71.2906,-156.7887,-16.58,63,0,0.00,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip?  75
What is the maximum temperature you would like for your trip?  90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp (F)"] <= max_temp) & \
                                       (city_data_df["Max Temp (F)"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Current Description
13,13,Ler,SS,8.3018,30.1418,81.55,30,100,1.88,overcast clouds
17,17,Rikitea,PF,-23.1203,-134.9692,78.24,79,100,8.58,overcast clouds
18,18,Bandarbeyla,SO,9.4942,50.8122,75.56,66,37,8.71,scattered clouds
22,22,Vaini,TO,-21.2000,-175.2000,75.22,89,100,6.34,moderate rain
23,23,Atuona,PF,-9.8000,-139.0333,79.03,72,43,7.98,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
694,694,Gigmoto,PH,13.7796,124.3900,79.21,82,80,1.88,broken clouds
698,698,Caravelas,BR,-17.7125,-39.2481,76.55,87,98,3.94,overcast clouds
699,699,Grand Gaube,MU,-20.0064,57.6608,76.37,87,77,1.34,broken clouds
704,704,Abonnema,NG,4.7084,6.7931,78.98,83,19,2.09,few clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp (F)           0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp (F)", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp (F),Current Description,Lat,Lng,Hotel Name
13,Ler,SS,81.55,overcast clouds,8.3018,30.1418,
17,Rikitea,PF,78.24,overcast clouds,-23.1203,-134.9692,
18,Bandarbeyla,SO,75.56,scattered clouds,9.4942,50.8122,
22,Vaini,TO,75.22,moderate rain,-21.2000,-175.2000,
23,Atuona,PF,79.03,scattered clouds,-9.8000,-139.0333,
24,Kapaa,US,77.18,clear sky,22.0752,-159.3190,
25,San Eduardo,PH,79.43,broken clouds,12.1679,125.4503,
30,Port Macquarie,AU,80.89,scattered clouds,-31.4333,152.9167,
35,Trat,TH,80.98,few clouds,12.5000,102.5000,
37,Saint-Philippe,RE,76.59,light rain,-21.3585,55.7679,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        pass
hotel_df

,City,Country,Max Temp (F),Current Description,Lat,Lng,Hotel Name
13,Ler,SS,81.55,overcast clouds,8.3018,30.1418,
17,Rikitea,PF,78.24,overcast clouds,-23.1203,-134.9692,People ThankYou
18,Bandarbeyla,SO,75.56,scattered clouds,9.4942,50.8122,JABIR HOTEL
22,Vaini,TO,75.22,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
23,Atuona,PF,79.03,scattered clouds,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
694,Gigmoto,PH,79.21,broken clouds,13.7796,124.3900,Cj's Travellers Inn
698,Caravelas,BR,76.55,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
699,Grand Gaube,MU,76.37,broken clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
704,Abonnema,NG,78.98,few clouds,4.7084,6.7931,Corper's Lodge Obonoma


In [13]:
clean_hotel_df = hotel_df.drop(hotel_df.index[hotel_df["Hotel Name"] == ""])
clean_hotel_df

,City,Country,Max Temp (F),Current Description,Lat,Lng,Hotel Name
17,Rikitea,PF,78.24,overcast clouds,-23.1203,-134.9692,People ThankYou
18,Bandarbeyla,SO,75.56,scattered clouds,9.4942,50.8122,JABIR HOTEL
22,Vaini,TO,75.22,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
23,Atuona,PF,79.03,scattered clouds,-9.8000,-139.0333,Villa Enata
24,Kapaa,US,77.18,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
694,Gigmoto,PH,79.21,broken clouds,13.7796,124.3900,Cj's Travellers Inn
698,Caravelas,BR,76.55,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
699,Grand Gaube,MU,76.37,broken clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
704,Abonnema,NG,78.98,few clouds,4.7084,6.7931,Corper's Lodge Obonoma


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt>
<dd>{City}</dd>
<dt>Country</dt>
<dd>{Country}</dd>
<dt>Weather Description</dt>
<dd>{Current Description}</dd>
<dt>Max Temp</dt>
<dd>{Max Temp (F)} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))